In [ ]:
import ee
import geemap
import pandas as pd

In [ ]:
# Initialize Google Earth Engine
ee.Initialize()


In [ ]:

# Retrieve and calculate the mean NDVI for a specific date range
ndvi = ee.ImageCollection('MODIS/061/MOD13Q1').filterDate('2023-04-05', '2023-04-14').select('NDVI').mean()

# Clip the NDVI data to a specific region defined by a feature collection (district boundary)
ndvi_clipped = ndvi.clip(ee.FeatureCollection('projects/sreekesh/assets/kodk'))

# Set visualization parameters for NDVI
ndviVis = {
    'min': 0,
    'max': 8000,
    'palette': [
        'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
        '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
        '012E01', '011D01', '011301'
    ]
}


In [ ]:

# Retrieve and calculate the mean LST for a specific date range
lst = ee.ImageCollection('JAXA/GCOM-C/L3/LAND/LST/V3').filterDate('2023-04-10', '2023-04-14').select('LST_AVE').mean()

# Clip the LST data to the same specific region (district boundary) as the NDVI data
lst_clipped = lst.clip(ee.FeatureCollection('projects/sreekesh/assets/kodk'))

# Calculate the minimum and maximum values of LST within the clipped region
lst_stats = lst_clipped.reduceRegion(ee.Reducer.minMax(), ee.FeatureCollection('projects/sreekesh/assets/kodk').geometry(), scale=500, maxPixels=1e13)
lst_min = ee.Number(lst_stats.get('LST_AVE_min'))
lst_max = ee.Number(lst_stats.get('LST_AVE_max'))

# Set visualization parameters for LST
lstVis = {
    'bands': ['LST_AVE'],
    'min': lst_min,
    'max': lst_max,
    'palette': [
        '040274', '040281', '0502a3', '0502b8', '0502ce', '0502e6',
        '0602ff', '235cb1', '307ef3', '269db1', '30c8e2', '32d3ef',
        '3be285', '3ff38f', '86e26f', '3ae237', 'b5e22e', 'd6e21f',
        'fff705', 'ffd611', 'ffb613', 'ff8b13', 'ff6e08', 'ff500d',
        'ff0000', 'de0101', 'c21301', 'a71001', '911003'
    ]
}

In [ ]:

# Retrieve and calculate the mean precipitation for a specific date range
precipitation = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").filterDate('2023-04-01', '2023-04-14').select('precipitation').mean()

# Clip the precipitation data to the same specific region (district boundary) as the other layers
precipitation_clipped = precipitation.clip(ee.FeatureCollection('projects/sreekesh/assets/kodk').geometry())

# Set visualization parameters for precipitation
precipitationVis = {
    'min': 0,
    'max': 12,
    'palette': ['blue', 'cyan', 'green', 'yellow', 'red']
}

In [ ]:



# Retrieve and calculate the mean climate data for a specific date range
climate = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE').filterDate('2020-03-01', '2020-04-14').mean()

# Clip the climate data to the same specific region (district boundary) as the other layers
climate_clipped = climate.clip(ee.FeatureCollection('projects/sreekesh/assets/kodk').geometry())

# Set visualization parameters for wind speed (vs band)
windVis = {
    'bands': ['vs'],  # Use the 'vs' band for wind speed
    'min': 0,
    'max': 292.3,  # Maximum wind speed value in the data
    'palette': ['blue', 'green', 'yellow', 'orange', 'red']  # Continuous wind speed color palette
}

In [ ]:


# Create a geemap object for the interactive map visualization
Map = geemap.Map()

# Set the center of the map and zoom level
Map.setCenter(6.746, 46.529, 2)

# Add the NDVI, LST, precipitation, and wind speed layers to the map with their respective visualization parameters
Map.addLayer(ndvi_clipped, ndviVis, 'Clipped NDVI')
Map.addLayer(lst_clipped, lstVis, 'Clipped LST')
Map.addLayer(precipitation_clipped, precipitationVis, 'Clipped Precipitation')
Map.addLayer(climate_clipped, windVis, 'Wind Speed')

# Display the map
Map

In [ ]:

# Combine layers for random forest input
combined = ee.Image([ndvi_clipped, lst_clipped, precipitation_clipped, climate_clipped])

# Load latitude and longitude points from a CSV file
csv_file_path = r"C:\Users\Asus\Desktop\TAMIL_N\A1234567\A.csv"
points_df = pd.read_csv(csv_file_path)

# Extract latitude and longitude columns from the DataFrame
latitudes = points_df['Longitude'].tolist()
longitudes = points_df['Latitude'].tolist()

# Create a list of points using the latitudes and longitudes
points = [ee.Geometry.Point(lon, lat) for lat, lon in zip(latitudes, longitudes)]

# Create a FeatureCollection from the list of points
point_collection = ee.FeatureCollection(points)

In [ ]:


# Sample the points from the FeatureCollection
sample_points = combined.sampleRegions(
    collection=point_collection,
    scale=100,
    geometries=True,
    tileScale=16
)

# Extract the properties and values from the sample
sample_values = sample_points.getInfo()["features"]
data = []
for feature in sample_values:
    properties = feature["properties"]
    coordinates = feature["geometry"]["coordinates"]
    data.append([coordinates[0], coordinates[1], properties["NDVI"], properties["LST_AVE"], properties["precipitation"], properties["vs"]])


In [ ]:

# Create a DataFrame from the extracted data
columns = ["Latitude", "Longitude", "NDVI", "LST", "Precipitation", "Wind"]
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame as a CSV file
csv_file_path_output = "D:\\new_sample_fixed_sir.csv"
df.to_csv(csv_file_path_output, index=False)


In [ ]:

# Display a success message
print("Sampled data saved as", csv_file_path_output)
